In [1]:
import warnings
warnings.simplefilter('ignore')

import pandas as pd
_TGT_WEEK_START = 8
_TGT_WEEK_END = _TGT_WEEK_START

## csv読み込み

In [2]:
# Read csv
games = pd.read_csv("../data/games.csv")
plays = pd.read_csv("../data/plays.csv")
players = pd.read_csv("../data/players.csv")
pffScoutingData = pd.read_csv("../data/pffScoutingData.csv")


## weekデータの読み込み、3frameごとのサンプリング、roleの追加、role番号の追加

In [3]:
_STEPSIZE = 3
pffScoutingData['nflId'] = pffScoutingData['nflId'].astype(float)
week_df_list = []
for week in range(_TGT_WEEK_START, _TGT_WEEK_END + 1):
    week_df = pd.read_csv("../data/week{}.csv".format(week))
    week_df = week_df[week_df['frameId'] % _STEPSIZE == 0]
    week_df = pd.merge(
        week_df,
        pffScoutingData[['gameId','playId','nflId','pff_role']],
        on=['gameId','playId','nflId'],
        how='left'
    )
    week_df = week_df[week_df['pff_role'].isin(['Pass', 'Pass Route', 'Pass Block', 'Pass Rush', 'Coverage'])]
    week_df_list.append(week_df)

week_df_all = pd.concat(week_df_list)
tmpdf = week_df_all.groupby(['gameId','playId'])[['frameId']].max().reset_index()
tmpdf['gameId_playId'] = tmpdf['gameId'].astype(str) + '_' + tmpdf['playId'].astype(str)
week_df_all['gameId_playId'] = week_df_all['gameId'].astype(str) + '_' + week_df_all['playId'].astype(str)
#pff_roleの追番を生成
week_df_all = week_df_all.sort_values(by=['gameId','playId','frameId','nflId'])
week_df_all['pff_role_num'] = week_df_all.groupby(['gameId','playId','frameId','pff_role']).cumcount()+1
week_df_all['pff_role_num'] = week_df_all['pff_role_num'].astype(int)
week_df_all['pff_role_num'] = week_df_all['pff_role'] + '_#' + week_df_all['pff_role_num'].astype(str)
print(len(week_df_all))
week_df_all.head()


304634


,gameId,playId,nflId,frameId,time,jerseyNumber,team,playDirection,x,y,s,a,dis,o,dir,event,pff_role,gameId_playId,pff_role_num
0,2021102800,189,37077.0,3,2021-10-29 00:27:23,18.0,ARI,right,21.38,6.95,0.00,0.00,0.00,45.69,303.24,None,Pass Route,2021102800_189,Pass Route_#1
18,2021102800,189,38779.0,3,2021-10-29 00:27:23,68.0,ARI,right,21.19,20.26,0.15,0.12,0.03,120.46,273.26,None,Pass Block,2021102800_189,Pass Block_#1
36,2021102800,189,39965.0,3,2021-10-29 00:27:23,67.0,ARI,right,21.64,25.65,0.07,0.75,0.01,98.71,53.62,None,Pass Block,2021102800_189,Pass Block_#2
54,2021102800,189,39973.0,3,2021-10-29 00:27:23,10.0,ARI,right,20.83,36.72,0.00,0.00,0.00,97.61,207.18,None,Pass Route,2021102800_189,Pass Route_#2
72,2021102800,189,39983.0,3,2021-10-29 00:27:23,86.0,ARI,right,21.81,31.28,0.01,0.01,0.01,63.37,205.66,None,Pass Route,2021102800_189,Pass Route_#3


## playDescriptionの説明文解析によるPass receiverの特定

In [4]:
tmp = week_df_all[['gameId','playId','nflId','pff_role_num']].drop_duplicates()
print(len(tmp))
tmp['nflId'] = tmp['nflId'].fillna(-1)
tmp['nflId'] = tmp['nflId'].astype(int)
tmp = pd.merge(
    tmp,
    players[['nflId','displayName']],
    on='nflId',
    how='left'
)
print(len(tmp))
tmp = tmp.dropna(subset='displayName')
print(len(tmp))
tmp['displayName_1'] = tmp['displayName'].str.split(' ', expand=True)[0]
tmp['displayName_2'] = tmp['displayName'].str.split(' ', expand=True)[1]
tmp['short_name'] = tmp['displayName_1'].str[0] + '.' + tmp['displayName_2']
tmp['short_name'] = ' ' + tmp['short_name']
tmp = pd.merge(
    tmp,
    plays[['gameId','playId','playDescription']],
    how='left',
    on=['gameId','playId']
)
print(len(tmp))
tmp = tmp[tmp['pff_role_num'].str.contains('Pass Route')]
print(len(tmp))
tmp['is_pass_receiver'] = tmp.apply(lambda x: x['short_name'] in x['playDescription'], axis=1)
tmp['is_pass_receiver'] = tmp['is_pass_receiver'].astype(int)
tmp_agg = tmp.groupby(['gameId','playId'])[['is_pass_receiver']].sum().reset_index().rename(columns={'is_pass_receiver': 'ttl_pass_receiver'})
tmp = pd.merge(
    tmp,
    tmp_agg,
    on=['gameId','playId'],
    how='left'
)
print(len(tmp))
print(len(tmp[tmp['ttl_pass_receiver'] > 1]), len(tmp[tmp['ttl_pass_receiver'] == 1]), len(tmp[tmp['ttl_pass_receiver'] == 0]))
tmp.to_csv('confirm_err_tmp.csv', index=False)
# tmp[tmp['ttl_pass_receiver'] != 1].to_csv('confirm_err_tmp.csv', index=False)
# tmp = tmp[tmp['ttl_pass_receiver'] <= 1] #15rows(3play, 2021090900_4653, 2021091201_901, 2021091207_2503)は分析の対象から除いてしまう
tmp = tmp[tmp['is_pass_receiver'] == 1]
print(len(tmp))

tmp = tmp[
    ['gameId','playId','nflId','pff_role_num','displayName','short_name']].rename(
    columns={'nflId': 'PassReceiver_nflId',
             'pff_role_num': 'PassReceiver_pff_role_num',
             'displayName': 'PassReceiver_displayName',
             'short_name': 'PassReceiver_short_name'}
)

plays['gameId_playId'] = plays['gameId'].astype(str) + '_' + plays['playId'].astype(str)
plays['is_pass_failed'] = (~(plays['passResult'] == 'C')).astype(int)
tgt_week_plays = plays[plays['gameId_playId'].isin(list(set(week_df_all['gameId_playId'].values)))]
dfagg = pd.merge(
    tgt_week_plays[['gameId','playId','is_pass_failed','passResult','playResult','playDescription']],
    tmp,
    on=['gameId','playId'],
    how='left'
)
# dfagg.to_csv('confirm_err_dfagg.csv', index=False)
for col in ['PassReceiver_pff_role_num','PassReceiver_displayName','PassReceiver_short_name']:
    dfagg[col] = dfagg[col].fillna('')
print('Anomaly play: ', len(dfagg[(dfagg['is_pass_failed'] == 0) & (dfagg['PassReceiver_pff_role_num']=='')]))
dfagg.head()

22704
22704
22704
22726
4766
4766
13 3941 812
857
Anomaly play:  18


,gameId,playId,is_pass_failed,passResult,playResult,playDescription,PassReceiver_nflId,PassReceiver_pff_role_num,PassReceiver_displayName,PassReceiver_short_name
0,2021102800,189,1,S,15,(12:58) (Shotgun) K.Murray sacked at ARI 2 for...,NaN,,,
1,2021102800,271,1,I,0,(11:30) (Shotgun) K.Murray pass incomplete dee...,NaN,,,
2,2021102800,334,1,I,0,(11:09) (Shotgun) Aa.Rodgers pass incomplete s...,NaN,,,
3,2021102800,368,0,C,7,(11:06) Aa.Rodgers pass short left to Am.Rodge...,NaN,,,
4,2021102800,392,1,I,0,(10:18) (Shotgun) Aa.Rodgers pass incomplete d...,44995.0,Pass Route_#1,Aaron Jones,A.Jones


## フィールド矩形dfの生成

In [5]:
field_rec_df = week_df[week_df['pff_role'].isin(['Pass','Pass Block', 'Pass Rush'])].groupby(['gameId','playId']).agg({
    'x': ['max', 'min'],
    'y': ['max', 'min']
}).reset_index()
field_rec_df.columns = ['gameId','playId','x_max','x_min','y_max','y_min']

field_df = {
    'gameId': [],
    'playId': [],
    'point_x': [],
    'point_y': []
}

def make_field_df(x):
    global field_df
    for point_x in range(int(x['x_min'] * 100), int(x['x_max'] * 100) + 100, 100):
        for point_y in range(int(x['y_min'] * 100), int(x['y_max'] * 100) + 100, 100):
            field_df['gameId'].append(x['gameId'])
            field_df['playId'].append(x['playId'])
            field_df['point_x'].append(point_x / 100.0)
            field_df['point_y'].append(point_y / 100.0)
    return x
field_rec_df = field_rec_df.apply(make_field_df, axis=1)
field_df = pd.DataFrame(field_df)
field_df['gameId'] = field_df['gameId'].astype(int)
field_df['playId'] = field_df['playId'].astype(int)
field_df_exp = pd.merge(
    week_df_all[['gameId','playId','frameId']].drop_duplicates(),
    field_df,
    on=['gameId','playId'],
    how='left'
)
field_df_exp['gameId_playId'] = field_df_exp['gameId'].astype(str) + '_' + field_df_exp['playId'].astype(str)
field_df_exp.head()

,gameId,playId,frameId,point_x,point_y,gameId_playId
0,2021102800,189,3,10.48,17.17,2021102800_189
1,2021102800,189,3,10.48,18.17,2021102800_189
2,2021102800,189,3,10.48,19.17,2021102800_189
3,2021102800,189,3,10.48,20.17,2021102800_189
4,2021102800,189,3,10.48,21.17,2021102800_189


## frameIdをキーにして、フィールド矩形dfに時刻frameIdにおけるプレイヤー状態をマージ

In [6]:
hoge = pd.pivot_table(week_df_all, 
                      index=['gameId_playId','frameId'], 
                      columns='pff_role_num', values=['x','y','s','a','dir','o','nflId']).reset_index()
colset = []
for row in hoge.columns:
    if row[1] == '':
        colset.append(row[0])
    else:
        colset.append(row[1] + '_' + row[0])
hoge.columns = colset
field_df_exp = pd.merge(
    field_df_exp,
    hoge,
    on=['gameId_playId','frameId'],
    how='left'
)
field_df_exp.head()

,gameId,playId,frameId,point_x,point_y,gameId_playId,Coverage_#1_a,Coverage_#2_a,Coverage_#3_a,Coverage_#4_a,...,Pass Route_#4_y,Pass Route_#5_y,Pass Rush_#1_y,Pass Rush_#2_y,Pass Rush_#3_y,Pass Rush_#4_y,Pass Rush_#5_y,Pass Rush_#6_y,Pass Rush_#7_y,Pass_#1_y
0,2021102800,189,3,10.48,17.17,2021102800_189,0.65,0.93,0.55,0.04,...,16.51,43.04,29.77,26.46,21.36,17.17,NaN,NaN,NaN,24.05
1,2021102800,189,3,10.48,18.17,2021102800_189,0.65,0.93,0.55,0.04,...,16.51,43.04,29.77,26.46,21.36,17.17,NaN,NaN,NaN,24.05
2,2021102800,189,3,10.48,19.17,2021102800_189,0.65,0.93,0.55,0.04,...,16.51,43.04,29.77,26.46,21.36,17.17,NaN,NaN,NaN,24.05
3,2021102800,189,3,10.48,20.17,2021102800_189,0.65,0.93,0.55,0.04,...,16.51,43.04,29.77,26.46,21.36,17.17,NaN,NaN,NaN,24.05
4,2021102800,189,3,10.48,21.17,2021102800_189,0.65,0.93,0.55,0.04,...,16.51,43.04,29.77,26.46,21.36,17.17,NaN,NaN,NaN,24.05


## playDirectionのマージ

In [7]:
print(len(field_df_exp))
field_df_exp_dir = pd.merge(
    field_df_exp,
    week_df_all[['gameId_playId','frameId','playDirection']].drop_duplicates(),
    on=['gameId_playId','frameId'],
    how='inner'
)
print(len(field_df_exp_dir))

2722182
2722182


## パスルーター、カバレッジの情報をdfaggに付与

In [8]:
dfagg_route = pd.merge(
    field_df_exp_dir[['gameId','playId','frameId']].drop_duplicates(),
    dfagg,
    on=['gameId','playId'],
    how='left'
)
dfagg_route = pd.merge(
    dfagg_route,
    field_df_exp_dir[['gameId','playId','frameId','playDirection','Pass_#1_x','Pass_#1_y'] + \
       [col for col in field_df_exp_dir.columns if 'Coverage' in col] + \
       [col for col in field_df_exp_dir.columns if 'Route' in col]].drop_duplicates(),
    on=['gameId','playId','frameId'],
    how='inner'
)
dfagg_route.head()

,gameId,playId,frameId,is_pass_failed,passResult,playResult,playDescription,PassReceiver_nflId,PassReceiver_pff_role_num,PassReceiver_displayName,...,Pass Route_#1_x,Pass Route_#2_x,Pass Route_#3_x,Pass Route_#4_x,Pass Route_#5_x,Pass Route_#1_y,Pass Route_#2_y,Pass Route_#3_y,Pass Route_#4_y,Pass Route_#5_y
0,2021102800,189,3,1,S,15,(12:58) (Shotgun) K.Murray sacked at ARI 2 for...,NaN,,,...,21.38,20.83,21.81,22.20,20.44,6.95,36.72,31.28,16.51,43.04
1,2021102800,189,6,1,S,15,(12:58) (Shotgun) K.Murray sacked at ARI 2 for...,NaN,,,...,21.39,20.84,21.86,22.20,20.37,6.94,36.72,31.27,16.50,43.01
2,2021102800,189,9,1,S,15,(12:58) (Shotgun) K.Murray sacked at ARI 2 for...,NaN,,,...,21.51,20.94,22.13,22.30,20.43,6.96,36.75,31.26,16.43,43.00
3,2021102800,189,12,1,S,15,(12:58) (Shotgun) K.Murray sacked at ARI 2 for...,NaN,,,...,22.14,21.41,22.87,22.84,20.64,7.03,36.89,31.28,16.12,42.92
4,2021102800,189,15,1,S,15,(12:58) (Shotgun) K.Murray sacked at ARI 2 for...,NaN,,,...,23.33,22.36,24.07,23.82,21.06,7.13,37.23,31.36,15.62,42.60


## データ量削減のため、フィールド矩形dfからはルーター、カバレッジの情報をdrop

In [9]:
field_df_exp_dir = field_df_exp_dir.drop(columns=[col for col in field_df_exp_dir.columns if 'Coverage' in col])
field_df_exp_dir = field_df_exp_dir.drop(columns=[col for col in field_df_exp_dir.columns if 'Route' in col])
field_df_exp_dir.head()

,gameId,playId,frameId,point_x,point_y,gameId_playId,Pass Block_#1_a,Pass Block_#2_a,Pass Block_#3_a,Pass Block_#4_a,...,Pass Block_#8_y,Pass Rush_#1_y,Pass Rush_#2_y,Pass Rush_#3_y,Pass Rush_#4_y,Pass Rush_#5_y,Pass Rush_#6_y,Pass Rush_#7_y,Pass_#1_y,playDirection
0,2021102800,189,3,10.48,17.17,2021102800_189,0.12,0.75,0.0,0.07,...,NaN,29.77,26.46,21.36,17.17,NaN,NaN,NaN,24.05,right
1,2021102800,189,3,10.48,18.17,2021102800_189,0.12,0.75,0.0,0.07,...,NaN,29.77,26.46,21.36,17.17,NaN,NaN,NaN,24.05,right
2,2021102800,189,3,10.48,19.17,2021102800_189,0.12,0.75,0.0,0.07,...,NaN,29.77,26.46,21.36,17.17,NaN,NaN,NaN,24.05,right
3,2021102800,189,3,10.48,20.17,2021102800_189,0.12,0.75,0.0,0.07,...,NaN,29.77,26.46,21.36,17.17,NaN,NaN,NaN,24.05,right
4,2021102800,189,3,10.48,21.17,2021102800_189,0.12,0.75,0.0,0.07,...,NaN,29.77,26.46,21.36,17.17,NaN,NaN,NaN,24.05,right


## passerが誰なのかの情報をplayresultに追加

In [10]:
hoge = week_df_all[['gameId','playId','nflId','pff_role']].drop_duplicates()
hoge['pff_role'] = hoge['pff_role'].astype(str)
hoge = hoge[hoge['pff_role']=='Pass']
hoge['nflId'] = hoge['nflId'].astype(int)
hoge.head()
print(len(hoge))
hoge = pd.merge(
    hoge,
    players[['nflId','displayName']],
    on='nflId',
    how='left'
)
print(len(hoge))
hoge
dfagg = pd.merge(
    dfagg,
    hoge[['gameId','playId','nflId','displayName']].rename(columns={'nflId': 'Passer_nflId', 'displayName': 'Passer_name'}),
    on=['gameId','playId'],
    how='left'
)
dfagg.head()

1032
1032


,gameId,playId,is_pass_failed,passResult,playResult,playDescription,PassReceiver_nflId,PassReceiver_pff_role_num,PassReceiver_displayName,PassReceiver_short_name,Passer_nflId,Passer_name
0,2021102800,189,1,S,15,(12:58) (Shotgun) K.Murray sacked at ARI 2 for...,NaN,,,,47784,Kyler Murray
1,2021102800,271,1,I,0,(11:30) (Shotgun) K.Murray pass incomplete dee...,NaN,,,,47784,Kyler Murray
2,2021102800,334,1,I,0,(11:09) (Shotgun) Aa.Rodgers pass incomplete s...,NaN,,,,29851,Aaron Rodgers
3,2021102800,368,0,C,7,(11:06) Aa.Rodgers pass short left to Am.Rodge...,NaN,,,,29851,Aaron Rodgers
4,2021102800,392,1,I,0,(10:18) (Shotgun) Aa.Rodgers pass incomplete d...,44995.0,Pass Route_#1,Aaron Jones,A.Jones,29851,Aaron Rodgers


## パスを投げる瞬間のframeIdを取得しdfagg_routeにマージ
 - speedのmaxをとるとsack時などにおかしくなる。最終行から６行目を取れば、それがイベント

In [11]:
# week_df_football = week_df[week_df['team'] == 'football']
# week_df_football = week_df_football.loc[week_df_football.groupby(['gameId','playId']).s.idxmax()].reset_index(drop=True).rename(columns={
#     'frameId': 'pass_throw_frameId'
# })[['gameId','playId','pass_throw_frameId']]
# print(len(dfagg_route))
# dfagg_route = pd.merge(
#     dfagg_route,
#     week_df_football,
#     on=['gameId','playId'],
#     how='left'
# )
# print(len(dfagg_route))
# dfagg_route.head()

passframe = week_df.groupby(['gameId','playId'])[['frameId']].max().reset_index().rename(columns={'frameId': 'pass_throw_frameId'})
passframe['pass_throw_frameId'] = passframe['pass_throw_frameId'] - (6 // _STEPSIZE) * _STEPSIZE
print(len(dfagg_route))
dfagg_route = pd.merge(
    dfagg_route,
    passframe,
    on=['gameId','playId'],
    how='left'
)
print(len(dfagg_route))
dfagg_route.head()



13892
13892


,gameId,playId,frameId,is_pass_failed,passResult,playResult,playDescription,PassReceiver_nflId,PassReceiver_pff_role_num,PassReceiver_displayName,...,Pass Route_#2_x,Pass Route_#3_x,Pass Route_#4_x,Pass Route_#5_x,Pass Route_#1_y,Pass Route_#2_y,Pass Route_#3_y,Pass Route_#4_y,Pass Route_#5_y,pass_throw_frameId
0,2021102800,189,3,1,S,15,(12:58) (Shotgun) K.Murray sacked at ARI 2 for...,NaN,,,...,20.83,21.81,22.20,20.44,6.95,36.72,31.28,16.51,43.04,48
1,2021102800,189,6,1,S,15,(12:58) (Shotgun) K.Murray sacked at ARI 2 for...,NaN,,,...,20.84,21.86,22.20,20.37,6.94,36.72,31.27,16.50,43.01,48
2,2021102800,189,9,1,S,15,(12:58) (Shotgun) K.Murray sacked at ARI 2 for...,NaN,,,...,20.94,22.13,22.30,20.43,6.96,36.75,31.26,16.43,43.00,48
3,2021102800,189,12,1,S,15,(12:58) (Shotgun) K.Murray sacked at ARI 2 for...,NaN,,,...,21.41,22.87,22.84,20.64,7.03,36.89,31.28,16.12,42.92,48
4,2021102800,189,15,1,S,15,(12:58) (Shotgun) K.Murray sacked at ARI 2 for...,NaN,,,...,22.36,24.07,23.82,21.06,7.13,37.23,31.36,15.62,42.60,48


## データ保存

In [12]:
# field_df_exp_dir.to_csv('../generated_data/field_df_week{}_v5.csv'.format(_TGT_WEEK_START), index=False)
# dfagg.to_csv('../generated_data/playresult_week{}_v5.csv'.format(_TGT_WEEK_START), index=False)
# dfagg_route.to_csv('../generated_data/dfagg_route_week{}_v5.csv'.format(_TGT_WEEK_START), index=False)

## モデル前処理

## データ取り込み

In [13]:
import pandas as pd
import numpy as np
import plotly.express as px
from scipy import stats

play_result = dfagg
df = field_df_exp_dir
dfagg_route = dfagg_route
display(dfagg_route.head(3))

# rusher_threatを無効化するためにはspeed, acceleration = 0, x, yは十分に遠い（=(0,0)ところにする
blockers_col = list(set([col[:-2] for col in df.columns if 'Block' in col and '_d' not in col and 'nfl' not in col]))
rushers_col = list(set([col[:-2] for col in df.columns if 'Rush' in col and '_d' not in col and 'nfl' not in col]))
coverages_col = list(set([col[:-2] for col in dfagg_route.columns if 'Coverage' in col and '_d' not in col and 'nfl' not in col]))
routers_col = list(set([col[:-2] for col in dfagg_route.columns if 'Route' in col and '_d' not in col and 'nfl' not in col]))

df = df.fillna(0)
dfagg_route = dfagg_route.fillna(0)
tmp = df.dropna()
print('detect Null {}'.format(len(df) - len(tmp)))

,gameId,playId,frameId,is_pass_failed,passResult,playResult,playDescription,PassReceiver_nflId,PassReceiver_pff_role_num,PassReceiver_displayName,...,Pass Route_#2_x,Pass Route_#3_x,Pass Route_#4_x,Pass Route_#5_x,Pass Route_#1_y,Pass Route_#2_y,Pass Route_#3_y,Pass Route_#4_y,Pass Route_#5_y,pass_throw_frameId
0,2021102800,189,3,1,S,15,(12:58) (Shotgun) K.Murray sacked at ARI 2 for...,NaN,,,...,20.83,21.81,22.2,20.44,6.95,36.72,31.28,16.51,43.04,48
1,2021102800,189,6,1,S,15,(12:58) (Shotgun) K.Murray sacked at ARI 2 for...,NaN,,,...,20.84,21.86,22.2,20.37,6.94,36.72,31.27,16.50,43.01,48
2,2021102800,189,9,1,S,15,(12:58) (Shotgun) K.Murray sacked at ARI 2 for...,NaN,,,...,20.94,22.13,22.3,20.43,6.96,36.75,31.26,16.43,43.00,48


detect Null 0


## ラッシャー影響度の計算

In [14]:

"""
ラインマン起因リスク - 向考慮あり、正規分布・ガンマ分布混合バージョン
ラッシャー：パサー方向に向かって危険度を持っている。
ブロッカー：パサー逆方向に向かって影響度を持っている
"""

# 1次元ガウス分布の平均パラメータを指定
m = 0.0

# 1次元ガウス分布の精度パラメータの係数を指定 βはμの精度(分散)に影響するので、βが大きくなるに従って、μ(x軸)における確率密度が大きい範囲が狭くなり(山が尖り)ます。
beta = 1
# ガンマ分布のパラメータを指定
a = 9.0 # aが大きくなるに従って、λ(y軸)が大きいほど確率密度が大きくなります。
b = 5.0 #bが大きくなるに従って、λ(y軸)が小さい(0に近い)ほど確率密度が大きくなります。


df['ruhsers_ttl_impact'] = 0    
for rusher_col in rushers_col:
    if '_d' in rusher_col or '_o' in rusher_col:
        continue
    print(rusher_col)
    df['vec_pnt2rshr_x'] = df['point_x'] - df[rusher_col + '_x']
    df['vec_pnt2rshr_y'] = df['point_y'] - df[rusher_col + '_y']
    direction_rad = np.radians(df[rusher_col + '_dir'].values)
    # pointを上を向いている場合の座標に変換
    df['vec_pnt2rshr_x_rotate'] = df['vec_pnt2rshr_x'].values * np.cos(direction_rad) - \
                                  df['vec_pnt2rshr_y'].values * np.sin(direction_rad)
    df['vec_pnt2rshr_y_rotate'] = df['vec_pnt2rshr_x'].values * np.sin(direction_rad) + \
                                  df['vec_pnt2rshr_y'].values * np.cos(direction_rad)
    #rusher_pointを原点[0,1.634]とした場合のpoint_fixの相対座標
    df['vec_pnt2rshr_y_rotate'] = df['vec_pnt2rshr_y_rotate'] + (1.69 * 3)
    df['vec_pnt2rshr_y_rotate'] = df['vec_pnt2rshr_y_rotate'].fillna(-1)
    #影響度の計算
    impct_norm = stats.norm.pdf(
        x=df['vec_pnt2rshr_x_rotate'].values / 3, 
        loc=m, 
        scale=np.sqrt(1.0 / beta / (df['vec_pnt2rshr_y_rotate'].values / 3))
    )
    impct_gamma = stats.gamma.pdf(x=df['vec_pnt2rshr_y_rotate'].values / 3, a=a, scale=1.0 / b)
    impct = impct_norm * impct_gamma
#     df[rusher_col + '_impact'] = impct * (df[rusher_col + '_s'].values + df[rusher_col + '_a'].values)
    df[rusher_col + '_impact'] = impct
    df[rusher_col + '_impact'] = df[rusher_col + '_impact'].fillna(0)
    df['ruhsers_ttl_impact'] = df['ruhsers_ttl_impact'] + df[rusher_col + '_impact']
print('ruhsers_ttl_impact...DONE')
    
df.head()

Pass Rush_#2
Pass Rush_#5
Pass Rush_#6
Pass Rush_#1
Pass Rush_#3
Pass Rush_#4
Pass Rush_#7
ruhsers_ttl_impact...DONE


,gameId,playId,frameId,point_x,point_y,gameId_playId,Pass Block_#1_a,Pass Block_#2_a,Pass Block_#3_a,Pass Block_#4_a,...,vec_pnt2rshr_y,vec_pnt2rshr_x_rotate,vec_pnt2rshr_y_rotate,Pass Rush_#2_impact,Pass Rush_#5_impact,Pass Rush_#6_impact,Pass Rush_#1_impact,Pass Rush_#3_impact,Pass Rush_#4_impact,Pass Rush_#7_impact
0,2021102800,189,3,10.48,17.17,2021102800_189,0.12,0.75,0.0,0.07,...,17.17,10.48,22.24,3.582892e-22,8.681060e-28,8.681060e-28,1.462709e-08,2.430455e-06,2.938006e-07,8.681060e-28
1,2021102800,189,3,10.48,18.17,2021102800_189,0.12,0.75,0.0,0.07,...,18.17,10.48,23.24,1.470197e-19,3.117454e-29,3.117454e-29,3.144912e-08,4.132280e-06,1.396379e-06,3.117454e-29
2,2021102800,189,3,10.48,19.17,2021102800_189,0.12,0.75,0.0,0.07,...,19.17,10.48,24.24,4.314341e-17,1.102012e-30,1.102012e-30,4.490702e-08,3.699062e-06,3.892057e-06,1.102012e-30
3,2021102800,189,3,10.48,20.17,2021102800_189,0.12,0.75,0.0,0.07,...,20.17,10.48,25.24,8.947811e-15,3.839586e-32,3.839586e-32,4.428932e-08,1.788985e-06,6.175397e-06,3.839586e-32
4,2021102800,189,3,10.48,21.17,2021102800_189,0.12,0.75,0.0,0.07,...,21.17,10.48,26.24,1.287627e-12,1.320029e-33,1.320029e-33,3.137347e-08,4.796757e-07,5.414313e-06,1.320029e-33


## ブロッカー to パサー影響度とブロッカー to ラッシャー影響度計算

In [15]:

df['blockers_ttl_impact'] = 0
df['blockers_ttl_impact_to_rusher'] = 0
for blocker_col in blockers_col:
    if '_d' in blocker_col or '_o' in blocker_col:
        continue
    print(blocker_col)
    df['vec_pnt2blckr_x'] = df['point_x'] - df[blocker_col + '_x']
    df['vec_pnt2blckr_y'] = df['point_y'] - df[blocker_col + '_y']
    direction_rad = np.radians(df[blocker_col + '_o'].values + 180.)
    # pointを上を向いている場合の座標に変換
    df['vec_pnt2blckr_x_rotate'] = df['vec_pnt2blckr_x'].values * np.cos(direction_rad) - \
                                   df['vec_pnt2blckr_y'].values * np.sin(direction_rad)
    df['vec_pnt2blckr_y_rotate'] = df['vec_pnt2blckr_x'].values * np.sin(direction_rad) + \
                                   df['vec_pnt2blckr_y'].values * np.cos(direction_rad)
    #blocker_pointを原点[0,1.634]とした場合のpoint_fixの相対座標
    df['vec_pnt2blckr_y_rotate'] = df['vec_pnt2blckr_y_rotate'] + (1.69 * 3)
    df['vec_pnt2blckr_y_rotate'] = df['vec_pnt2blckr_y_rotate'].fillna(-1)
    #影響度の計算
    impct_norm = stats.norm.pdf(
        x=df['vec_pnt2blckr_x_rotate'].values / 3, 
        loc=m, 
        scale=np.sqrt(1.0 / beta / (df['vec_pnt2blckr_y_rotate'].values / 3))
    )
    impct_gamma = stats.gamma.pdf(x=df['vec_pnt2blckr_y_rotate'].values / 3, a=a, scale=1.0 / b)
    impct = impct_norm * impct_gamma
#     df[blocker_col + '_impact'] = impct * (df[blocker_col + '_s'].values + df[blocker_col + '_a'].values)
    df[blocker_col + '_impact'] = impct
    df[blocker_col + '_impact'] = df[blocker_col + '_impact'].fillna(0)
    df['blockers_ttl_impact'] = df['blockers_ttl_impact'] + df[blocker_col + '_impact']
    
    #ここからblckr to rusherの影響度（blckrの影響度が前を向いているバージョン
    # pointを上を向いている場合の座標に変換
    direction_rad = np.radians(df[blocker_col + '_o'].values)
    df['vec_pnt2blckr_x_rotate'] = df['vec_pnt2blckr_x'].values * np.cos(direction_rad) - \
                                  df['vec_pnt2blckr_y'].values * np.sin(direction_rad)
    df['vec_pnt2blckr_y_rotate'] = df['vec_pnt2blckr_x'].values * np.sin(direction_rad) + \
                                  df['vec_pnt2blckr_y'].values * np.cos(direction_rad)
    #rusher_pointを原点[0,1.634]とした場合のpoint_fixの相対座標
    df['vec_pnt2blckr_y_rotate'] = df['vec_pnt2blckr_y_rotate'] + (1.69 * 3)
    df['vec_pnt2blckr_y_rotate'] = df['vec_pnt2blckr_y_rotate'].fillna(-1)
    #影響度の計算
    impct_norm = stats.norm.pdf(
        x=df['vec_pnt2blckr_x_rotate'].values / 3, 
        loc=m, 
        scale=np.sqrt(1.0 / beta / (df['vec_pnt2blckr_y_rotate'].values / 3))
    )
    impct_gamma = stats.gamma.pdf(x=df['vec_pnt2blckr_y_rotate'].values / 3, a=a, scale=1.0 / b)
    impct = impct_norm * impct_gamma
    df[blocker_col + '_impact_to_rshr'] = impct
    df[blocker_col + '_impact_to_rshr'] = df[blocker_col + '_impact_to_rshr'].fillna(0)
    df['blockers_ttl_impact_to_rusher'] = df['blockers_ttl_impact_to_rusher'] + df[blocker_col + '_impact_to_rshr']

    
print('blockers_ttl_impact...DONE')

df['dist_from_passer'] = np.sqrt((df['point_x'] - df['Pass_#1_x']) ** 2 + (df['point_y'] - df['Pass_#1_y']) ** 2)
df['ttl_point_risk'] = df['ruhsers_ttl_impact'] - df['blockers_ttl_impact']
# risk０未満は０に丸め込み
df.loc[df['ttl_point_risk'] < 0, 'ttl_point_risk'] = 0
dfagg = df.loc[df.groupby(['gameId','playId','frameId']).dist_from_passer.idxmin()].reset_index(drop=True)
tgt_cols = [
    'gameId','playId','frameId','point_x','point_y',
    'Pass Block_#1_nflId','Pass Block_#2_nflId','Pass Block_#3_nflId','Pass Block_#4_nflId',
    'Pass Block_#5_nflId','Pass Block_#6_nflId','Pass Block_#7_nflId','Pass Block_#8_nflId',
    'Pass Block_#9_nflId','Pass Rush_#1_nflId','Pass Rush_#2_nflId','Pass Rush_#3_nflId',
    'Pass Rush_#4_nflId','Pass Rush_#5_nflId','Pass Rush_#6_nflId','Pass Rush_#7_nflId',
    'Pass Rush_#3_impact','Pass Rush_#5_impact',
    'Pass Rush_#1_impact','Pass Rush_#4_impact',
    'Pass Rush_#6_impact','Pass Rush_#2_impact',
    'Pass Rush_#7_impact',
    'Pass Block_#1_impact',
    'Pass Block_#9_impact','Pass Block_#4_impact',
    'Pass Block_#7_impact','Pass Block_#8_impact',
    'Pass Block_#5_impact','Pass Block_#3_impact',
    'Pass Block_#2_impact','Pass Block_#6_impact',    
    'Pass Block_#1_impact_to_rshr',
    'Pass Block_#9_impact_to_rshr','Pass Block_#4_impact_to_rshr',
    'Pass Block_#7_impact_to_rshr','Pass Block_#8_impact_to_rshr',
    'Pass Block_#5_impact_to_rshr','Pass Block_#3_impact_to_rshr',
    'Pass Block_#2_impact_to_rshr','Pass Block_#6_impact_to_rshr','ttl_point_risk'
]
# Initialize columns
for col in tgt_cols:
    if col not in dfagg.columns:
        dfagg[col] = 0

dfagg = dfagg[tgt_cols].rename(columns={'ttl_point_risk': 'passer_point_risk'})
dfagg.head()

Pass Block_#2
Pass Block_#6
Pass Block_#5
Pass Block_#4
Pass Block_#7
Pass Block_#8
Pass Block_#1
Pass Block_#3
blockers_ttl_impact...DONE


,gameId,playId,frameId,point_x,point_y,Pass Block_#1_nflId,Pass Block_#2_nflId,Pass Block_#3_nflId,Pass Block_#4_nflId,Pass Block_#5_nflId,...,Pass Block_#1_impact_to_rshr,Pass Block_#9_impact_to_rshr,Pass Block_#4_impact_to_rshr,Pass Block_#7_impact_to_rshr,Pass Block_#8_impact_to_rshr,Pass Block_#5_impact_to_rshr,Pass Block_#3_impact_to_rshr,Pass Block_#2_impact_to_rshr,Pass Block_#6_impact_to_rshr,passer_point_risk
0,2021102800,189,3,17.48,24.17,38779.0,39965.0,42367.0,44949.0,52480.0,...,0.000000e+00,0,0.032441,4.707686e-84,4.707686e-84,0.012772,0.007654,0.000893,4.707686e-84,0.0
1,2021102800,189,6,17.48,24.17,38779.0,39965.0,42367.0,44949.0,52480.0,...,2.615168e-17,0,0.020517,4.707686e-84,4.707686e-84,0.010962,0.000624,0.000210,4.707686e-84,0.0
2,2021102800,189,9,17.48,24.17,38779.0,39965.0,42367.0,44949.0,52480.0,...,3.633487e-06,0,0.030180,4.707686e-84,4.707686e-84,0.005600,0.000526,0.000020,4.707686e-84,0.0
3,2021102800,189,12,17.48,24.17,38779.0,39965.0,42367.0,44949.0,52480.0,...,4.942804e-07,0,0.001539,4.707686e-84,4.707686e-84,0.007004,0.026885,0.000094,4.707686e-84,0.0
4,2021102800,189,15,17.48,24.17,38779.0,39965.0,42367.0,44949.0,52480.0,...,1.303494e-09,0,0.043665,4.707686e-84,4.707686e-84,0.008646,0.019134,0.000407,4.707686e-84,0.0


## ヒートマップ 可視化用データ保存

In [16]:
# df_game = df[(df['gameId']==2021090900) & (df['playId']==97)]
# df_game.to_csv('../generated_data/field_df_2021090900_97.csv', index=False)

## ラッシャー評価用データ保存

In [17]:
# dflineman = df.loc[df.groupby(['gameId','playId','frameId']).dist_from_passer.idxmin()].reset_index(drop=True)
# dflineman.to_csv('../generated_data/rusher_score_week{}_v6.csv'.format(_TGT_WEEK_START), index=False)
# dflineman.head(3)

## ラインマン評価用データ保存

In [18]:
# Initialize columns
for col in tgt_cols:
    if col not in df.columns:
        df[col] = 0
save_tgt_cols = [
    'gameId','playId','frameId','point_x','point_y',
    'Pass Block_#1_nflId','Pass Block_#2_nflId','Pass Block_#3_nflId',
    'Pass Block_#4_nflId','Pass Block_#5_nflId','Pass Block_#6_nflId',
    'Pass Block_#7_nflId','Pass Block_#8_nflId','Pass Block_#9_nflId',
    'Pass Rush_#1_nflId','Pass Rush_#2_nflId','Pass Rush_#3_nflId',
    'Pass Rush_#4_nflId','Pass Rush_#5_nflId','Pass Rush_#6_nflId',
    'Pass Rush_#7_nflId','Pass Rush_#4_impact','Pass Rush_#7_impact',
    'Pass Rush_#5_impact','Pass Rush_#6_impact','Pass Rush_#2_impact',
    'Pass Rush_#3_impact','Pass Rush_#1_impact','blockers_ttl_impact',
    'Pass Block_#6_impact','Pass Block_#2_impact','Pass Block_#1_impact',
    'Pass Block_#8_impact','Pass Block_#4_impact','Pass Block_#7_impact',
    'Pass Block_#3_impact','Pass Block_#5_impact','Pass Block_#9_impact',
    'dist_from_passer','ttl_point_risk'
]
# df[save_tgt_cols].to_csv('../generated_data/field_df_lineman_week{}_v6.csv'.format(_TGT_WEEK_START), index=False)

## カバレッジ影響度計算

In [19]:
# パスルート起因リスク


# 1次元ガウス分布の平均パラメータを指定
m = 0.0

# 1次元ガウス分布の精度パラメータの係数を指定 βはμの精度(分散)に影響するので、βが大きくなるに従って、μ(x軸)における確率密度が大きい範囲が狭くなり(山が尖り)ます。
beta = 2
# ガンマ分布のパラメータを指定
a = 9.0 # aが大きくなるに従って、λ(y軸)が大きいほど確率密度が大きくなります。
b = 5.0 #bが大きくなるに従って、λ(y軸)が小さい(0に近い)ほど確率密度が大きくなります。


print(len(dfagg_route))
dfagg_route['direction_val'] = (dfagg_route['playDirection'] == 'right').astype(int) * 2 - 1
dfagg_route['routers_ttl_risk'] = 0

for router_col in routers_col:
    if '_d' in router_col or '_o' in router_col:
        continue
    print(router_col)
    dfagg_route[router_col + '_ttl_risk'] = 0
    for coverage_col in coverages_col:
        if '_d' in coverage_col or '_o' in coverage_col:
            continue
        
        # coverage impact direction
        dfagg_route['vec_cvrg2rtr_x'] = dfagg_route[router_col + '_x'] - dfagg_route[coverage_col + '_x']
        dfagg_route['vec_cvrg2rtr_y'] = dfagg_route[router_col + '_y'] - dfagg_route[coverage_col + '_y']
        dfagg_route['vec_pssr2rtr_x'] = dfagg_route[router_col + '_x'] - dfagg_route['Pass_#1_x']
        dfagg_route['vec_pssr2rtr_y'] = dfagg_route[router_col + '_y'] - dfagg_route['Pass_#1_y']
        direction_rad = np.radians(
            np.rad2deg(np.arctan(dfagg_route['vec_pssr2rtr_y'] / dfagg_route['vec_pssr2rtr_x']) + 0)
        )        
        # coverage影響方向が上を向いている場合のルーターの相対座標に変換
        dfagg_route['vec_cvrg2rtr_x_rotate'] = dfagg_route['vec_cvrg2rtr_x'].values * np.cos(direction_rad) - \
                                               dfagg_route['vec_cvrg2rtr_y'].values * np.sin(direction_rad)
        dfagg_route['vec_cvrg2rtr_y_rotate'] = dfagg_route['vec_cvrg2rtr_x'].values * np.sin(direction_rad) + \
                                               dfagg_route['vec_cvrg2rtr_y'].values * np.cos(direction_rad)
        #coverageを原点[0,1.634]とした場合のpoint_fixの相対座標
        dfagg_route['vec_cvrg2rtr_y_rotate'] = dfagg_route['vec_cvrg2rtr_y_rotate'] + (1.69 * 3)
        dfagg_route['vec_cvrg2rtr_y_rotate'] = dfagg_route['vec_cvrg2rtr_y_rotate'].fillna(-1)
        
        #影響度の計算: coverage impact to pass_route
        impct_norm = stats.norm.pdf(
            x=dfagg_route['vec_cvrg2rtr_x_rotate'].values / 3, 
            loc=m, 
            scale=np.sqrt(1.0 / beta / (dfagg_route['vec_cvrg2rtr_y_rotate'].values / 3))
        )
        impct_gamma = stats.gamma.pdf(x=dfagg_route['vec_cvrg2rtr_y_rotate'].values / 3, a=a, scale=1.0 / b)
        dfagg_route['tmp'] = impct_norm * impct_gamma
        dfagg_route['tmp'] = dfagg_route['tmp'].fillna(0)
        dfagg_route[router_col + '_ttl_risk'] = dfagg_route[router_col + '_ttl_risk'] + dfagg_route['tmp']
    dist_from_passer = np.sqrt((dfagg_route['Pass_#1_x'] - dfagg_route[router_col + '_x']) ** 2 + \
                               (dfagg_route['Pass_#1_y'] - dfagg_route[router_col + '_y']) ** 2)
    dfagg_route[router_col + '_ttl_risk'] = dfagg_route[router_col + '_ttl_risk'].values * dist_from_passer
    dfagg_route['routers_ttl_risk'] = dfagg_route['routers_ttl_risk'] + dfagg_route[router_col + '_ttl_risk']
dfagg_route['PassReceiver_pff_role_num'] = dfagg_route['PassReceiver_pff_role_num'].astype(str)
def get_PassReceiver_ttl_risk(x):
    pass_receiver = x['PassReceiver_pff_role_num']
    if 'Pass Route_' in pass_receiver:
        x['PassReceiver_ttl_risk'] = x[pass_receiver + '_ttl_risk']
    else:
        x['PassReceiver_ttl_risk'] = x['routers_ttl_risk'] / 5.
    return x
dfagg_route = dfagg_route.apply(get_PassReceiver_ttl_risk, axis=1)
dfagg_route.head(3)

13892
Pass Route_#4
Pass Route_#5
Pass Route_#3
Pass Route_#2
Pass Route_#1


,gameId,playId,frameId,is_pass_failed,passResult,playResult,playDescription,PassReceiver_nflId,PassReceiver_pff_role_num,PassReceiver_displayName,...,vec_pssr2rtr_x,vec_pssr2rtr_y,vec_cvrg2rtr_x_rotate,vec_cvrg2rtr_y_rotate,tmp,Pass Route_#5_ttl_risk,Pass Route_#3_ttl_risk,Pass Route_#2_ttl_risk,Pass Route_#1_ttl_risk,PassReceiver_ttl_risk
0,2021102800,189,3,1,S,15,(12:58) (Shotgun) K.Murray sacked at ARI 2 for...,0.0,,,...,3.42,-17.10,11.007999,-14.531808,0.0,3.043555e-15,1.670558e-10,4.931881e-30,0.002561,0.076613
1,2021102800,189,6,1,S,15,(12:58) (Shotgun) K.Murray sacked at ARI 2 for...,0.0,,,...,3.46,-17.11,11.041996,-14.520049,0.0,6.864129e-13,6.268928e-10,6.737273e-31,0.002326,0.082880
2,2021102800,189,9,1,S,15,(12:58) (Shotgun) K.Murray sacked at ARI 2 for...,0.0,,,...,3.63,-17.07,11.281910,-14.521840,0.0,1.729221e-14,5.745347e-09,2.643706e-33,0.002253,0.094661


## パス実行時のリスク値の抽出

In [20]:
print(len(play_result))
play_result2 = pd.merge(
    play_result,
    dfagg_route[['gameId','playId','pass_throw_frameId']].drop_duplicates(),
    on=['gameId','playId'],
    how='left'
)
print(len(play_result2))
play_result2 = pd.merge(
    play_result2,
    dfagg_route[['gameId','playId','frameId','PassReceiver_ttl_risk','routers_ttl_risk']],
    left_on=['gameId','playId','pass_throw_frameId'],
    right_on=['gameId','playId','frameId'],
    how='left'
)
print(len(play_result2))

dfagg_res = pd.merge(
    dfagg,
    dfagg.groupby(['gameId','playId'])[['frameId']].mean().reset_index().rename(columns={'frameId': 'half_time'}),
    on=['gameId','playId'],
    how='left'
)
tgt_cols = [
    'Pass Block_#1_nflId','Pass Block_#2_nflId','Pass Block_#3_nflId','Pass Block_#4_nflId',
    'Pass Block_#5_nflId','Pass Block_#6_nflId','Pass Block_#7_nflId','Pass Block_#8_nflId',
    'Pass Block_#9_nflId','Pass Rush_#1_nflId','Pass Rush_#2_nflId','Pass Rush_#3_nflId',
    'Pass Rush_#4_nflId','Pass Rush_#5_nflId','Pass Rush_#6_nflId','Pass Rush_#7_nflId',
    'Pass Rush_#3_impact','Pass Rush_#5_impact',
    'Pass Rush_#1_impact','Pass Rush_#4_impact',
    'Pass Rush_#6_impact','Pass Rush_#2_impact',
    'Pass Rush_#7_impact','Pass Block_#1_impact',
    'Pass Block_#9_impact','Pass Block_#4_impact',
    'Pass Block_#7_impact','Pass Block_#8_impact',
    'Pass Block_#5_impact','Pass Block_#3_impact',
    'Pass Block_#2_impact','Pass Block_#6_impact',
    'Pass Block_#1_impact_to_rshr',
    'Pass Block_#9_impact_to_rshr','Pass Block_#4_impact_to_rshr',
    'Pass Block_#7_impact_to_rshr','Pass Block_#8_impact_to_rshr',
    'Pass Block_#5_impact_to_rshr','Pass Block_#3_impact_to_rshr',
    'Pass Block_#2_impact_to_rshr','Pass Block_#6_impact_to_rshr',
    'passer_point_risk'
]
dfagg_res = dfagg_res[dfagg_res['frameId'] > dfagg_res['half_time']].groupby(['gameId','playId'])[tgt_cols].mean().reset_index()

play_result2 = pd.merge(
    play_result2,
    dfagg_res,
    on=['gameId','playId'],
    how='left'
)
print(len(play_result2))
play_result2.head()

1036
1036
1050
1050


,gameId,playId,is_pass_failed,passResult,playResult,playDescription,PassReceiver_nflId,PassReceiver_pff_role_num,PassReceiver_displayName,PassReceiver_short_name,...,Pass Block_#1_impact_to_rshr,Pass Block_#9_impact_to_rshr,Pass Block_#4_impact_to_rshr,Pass Block_#7_impact_to_rshr,Pass Block_#8_impact_to_rshr,Pass Block_#5_impact_to_rshr,Pass Block_#3_impact_to_rshr,Pass Block_#2_impact_to_rshr,Pass Block_#6_impact_to_rshr,passer_point_risk
0,2021102800,189,1,S,15,(12:58) (Shotgun) K.Murray sacked at ARI 2 for...,NaN,,,,...,0.245716,0.0,0.010104,2.326377e-40,2.326377e-40,0.189693,0.005930,1.638748e-04,2.326377e-40,0.000000
1,2021102800,271,1,I,0,(11:30) (Shotgun) K.Murray pass incomplete dee...,NaN,,,,...,0.034359,0.0,0.000823,5.380089e-227,5.380089e-227,0.007649,0.017903,5.293550e-05,5.380089e-227,0.189768
2,2021102800,334,1,I,0,(11:09) (Shotgun) Aa.Rodgers pass incomplete s...,NaN,,,,...,0.018249,0.0,0.044095,0.000000e+00,0.000000e+00,0.004804,0.004722,2.484179e-02,0.000000e+00,0.000000
3,2021102800,368,0,C,7,(11:06) Aa.Rodgers pass short left to Am.Rodge...,NaN,,,,...,0.001264,0.0,0.032005,0.000000e+00,0.000000e+00,0.000004,0.000000,1.055646e-01,5.196547e-02,0.000000
4,2021102800,392,1,I,0,(10:18) (Shotgun) Aa.Rodgers pass incomplete d...,44995.0,Pass Route_#1,Aaron Jones,A.Jones,...,0.093338,0.0,0.046815,0.000000e+00,0.000000e+00,0.102150,0.010489,7.338136e-07,0.000000e+00,0.018518


## プレイヤーの身長・体重をマージ

In [21]:
players = pd.read_csv('../data/players.csv')

players['feet'] = players['height'].str.split('-', expand=True)[0].astype(int)
players['inch'] = players['height'].str.split('-', expand=True)[1].astype(int)
players['height_meter'] = (players['feet'] * 0.3048) + (players['inch'] * 0.0254) # 1feet: 0.3048m, 1inch: 0.0254m
players['weight_per_meter'] = players['weight'] / players['height_meter']
# players.to_csv('../generated_data/players_v6.csv', index=False)

# Pass Rush / Pass Blockのみを抽出してから標準化 全プレイヤーで標準化する必要あり

weed_df_rushers = []
weed_df_blckrs = []
for i in range(1, 8 + 1):
    week_df = pd.read_csv('../data/week{}.csv'.format(i))
    week_df = pd.merge(
        week_df,
        pffScoutingData[['gameId','playId','nflId','pff_role']],
        on=['gameId','playId','nflId'],
        how='left'
    )
    weed_df_rushers.append(week_df[(week_df['pff_role'] == 'Pass Rush') & (week_df['frameId'] == 1)])
    weed_df_blckrs.append(week_df[(week_df['pff_role'] == 'Pass Block') & (week_df['frameId'] == 1)])
weed_df_rushers = pd.concat(weed_df_rushers)
weed_df_blckrs = pd.concat(weed_df_blckrs)

all_rusher_nflId = list(weed_df_rushers['nflId'].astype(int).unique())
all_blckr_nflId = list(weed_df_blckrs['nflId'].astype(int).unique())

def min_max_normalization(players, week_df_players):
    # Min-Max Normalization
    players['height_meter'] = (players['height_meter'] - players['height_meter'].min()) / (players['height_meter'].max() - players['height_meter'].min())
    players['weight'] = (players['weight'] - players['weight'].min()) / (players['weight'].max() - players['weight'].min())
    players['weight_per_meter'] = (players['weight_per_meter'] - players['weight_per_meter'].min()) / (players['weight_per_meter'].max() - players['weight_per_meter'].min())
    week_df_players['nflId'] = week_df_players['nflId'].astype(int)
    ply_cnt = week_df_players.groupby('nflId')[['playId']].count().reset_index().rename(columns={'playId':'play_cnt'})
    players = pd.merge(
        players,
        ply_cnt,
        on='nflId',
        how='left'
    )
    return players
rush_players = min_max_normalization(players[players['nflId'].isin(all_rusher_nflId)], weed_df_rushers)
blck_players = min_max_normalization(players[players['nflId'].isin(all_blckr_nflId)], weed_df_blckrs)
weed_df_rushers = pd.merge(
    weed_df_rushers,
    rush_players[['nflId','displayName']],
    on='nflId',
    how='left'
)
weed_df_blckrs = pd.merge(
    weed_df_blckrs,
    blck_players[['nflId','displayName']],
    on='nflId',
    how='left'
)
weed_df_rushers.to_csv('../generated_data/week_df_rushers_v6.csv', index=False)
weed_df_blckrs.to_csv('../generated_data/week_df_blckrs_v6.csv', index=False)
rush_players.to_csv('../generated_data/rush_players_v6.csv', index=False)
blck_players.to_csv('../generated_data/blck_players_v6.csv', index=False)

for col in play_result2.columns:
    if 'nflId' in col:
        play_result2[col] = play_result2[col].fillna(0)
        play_result2[col] = play_result2[col].astype(int)

play_result3 = play_result2
for i in range(1, 7 + 1):
    play_result3 = pd.merge(
        play_result3,
        rush_players[['nflId','height_meter','weight','weight_per_meter']].rename(columns={
            'nflId': 'Pass Rush_#{}_nflId'.format(i),
            'height_meter': 'Pass Rush_#{}_height_meter'.format(i),
            'weight': 'Pass Rush_#{}_weight'.format(i),
            'weight_per_meter': 'Pass Rush_#{}_weight_per_meter'.format(i),
        }),
        on='Pass Rush_#{}_nflId'.format(i),
        how='left'
    )
for i in range(1, 9+1):
    play_result3 = pd.merge(
        play_result3,
        blck_players[['nflId','height_meter','weight']].rename(columns={
            'nflId': 'Pass Block_#{}_nflId'.format(i),
            'height_meter': 'Pass Block_#{}_height_meter'.format(i),
            'weight': 'Pass Block_#{}_weight'.format(i),
            'weight_per_meter': 'Pass Block_#{}_weight_per_meter'.format(i),
        }),
        on='Pass Block_#{}_nflId'.format(i),
        how='left'
    )
play_result3 = play_result3.fillna(0)
play_result3.head(2)

,gameId,playId,is_pass_failed,passResult,playResult,playDescription,PassReceiver_nflId,PassReceiver_pff_role_num,PassReceiver_displayName,PassReceiver_short_name,...,Pass Block_#5_height_meter,Pass Block_#5_weight,Pass Block_#6_height_meter,Pass Block_#6_weight,Pass Block_#7_height_meter,Pass Block_#7_weight,Pass Block_#8_height_meter,Pass Block_#8_weight,Pass Block_#9_height_meter,Pass Block_#9_weight
0,2021102800,189,1,S,15,(12:58) (Shotgun) K.Murray sacked at ARI 2 for...,0,,,,...,0.866667,0.655172,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2021102800,271,1,I,0,(11:30) (Shotgun) K.Murray pass incomplete dee...,0,,,,...,0.866667,0.655172,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 実際にパスを投げたプレー、ファールプレーにフラグを立てる

In [22]:
play = pd.read_csv('../data/plays.csv')
play = play[['gameId','playId','passResult','prePenaltyPlayResult','foulName1']]
play['is_foul_play'] = (~play['foulName1'].isnull()).astype(int)
# pass result: array(['I', 'C', 'S', 'R', 'IN'], dtype=object)
play['is_pass_done'] = play['passResult'].isin(['I', 'C', 'S', 'IN']).astype(int)
play_result3 = pd.merge(
    play_result3,
    play[['gameId','playId','is_pass_done','prePenaltyPlayResult','is_foul_play']],
    on=['gameId','playId'],
    how='left'
)
play_result3.head(2)

,gameId,playId,is_pass_failed,passResult,playResult,playDescription,PassReceiver_nflId,PassReceiver_pff_role_num,PassReceiver_displayName,PassReceiver_short_name,...,Pass Block_#6_weight,Pass Block_#7_height_meter,Pass Block_#7_weight,Pass Block_#8_height_meter,Pass Block_#8_weight,Pass Block_#9_height_meter,Pass Block_#9_weight,is_pass_done,prePenaltyPlayResult,is_foul_play
0,2021102800,189,1,S,15,(12:58) (Shotgun) K.Murray sacked at ARI 2 for...,0,,,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,-11,1
1,2021102800,271,1,I,0,(11:30) (Shotgun) K.Murray pass incomplete dee...,0,,,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0


## パサー責任パスミスプレーを除く
 - ラインマン からのリスクが低く、レシーバーのリスクも低いのに、パスが失敗している or パス成功するもヤードが下がっているプレー(苦し紛れのパス/判断ミスのパス)はパサー・レシーバー責任のミスと見做して解析の対象から除く
 - ファールプレーを除く
 - メモ）Sackになっている場合はPassReceiver_ttl_riskは全ルータの平均リスク


In [23]:
print(len(play_result3))
play_result3 = play_result3[play_result3['is_foul_play'] != 1]
print(len(play_result3))

# PassReceiver_ttl_risk 2未満、passer_point_risk< 0.001でpassResult='C'でないもの、playResult < 0のものを除く
play_result3 = play_result3[
    ~((play_result3['PassReceiver_ttl_risk'] < 2) &
    (play_result3['passer_point_risk'] < 0.001) &
    (play_result3['passResult']!='C'))
]
print(len(play_result3))
play_result3 = play_result3[
    ~((play_result3['PassReceiver_ttl_risk'] < 2) &
    (play_result3['passer_point_risk'] < 0.001) &
    (play_result3['passResult']=='C') &
    (play_result3['playResult'] <= 0) & 
    (play_result3['prePenaltyPlayResult'] <= 0))
]
print(len(play_result3))
# play_result3.loc[(play_result3['passResult']=='C') & (play_result3['prePenaltyPlayResult'] < 0), 'is_pass_failed'] = 1

1066
978
908
905


In [24]:
# play_result3.to_csv('tmp_v6_pre.csv', index=False)
play_result3.to_csv('../generated_data/model_week{}_v6.csv'.format(_TGT_WEEK_START), index=False)


## ブロッカー to ラッシャーの影響度の集計
 - プレー前半の影響度で計算する！

In [25]:
rusher_box_list = []
for i in range(1, 7 + 1):
    col = 'Pass Rush_#{}'.format(i)
    print(col)
    df['dist_to_rshr'] = np.sqrt(
        (df[col + '_x'] - df['point_x']) ** 2 + \
        (df[col + '_y'] - df['point_y']) ** 2
    )
    blocker_col = []
    for j in range(1,9 + 1):
        blocker_col.append('Pass Block_#{}_impact_to_rshr'.format(j))
        blocker_col.append('Pass Block_#{}_nflId'.format(j))
        
    tmp = df[['gameId','playId','frameId', col + '_nflId', 'blockers_ttl_impact_to_rusher','dist_to_rshr'] + blocker_col].rename(columns={
        col + '_nflId': 'Pass Rush_nflId',
        col + '_impact': 'Pass Rush_impact'
    })
    tmp = tmp.loc[tmp.groupby(['gameId','playId','frameId']).dist_to_rshr.idxmin()].reset_index(drop=True)
    rusher_box_list.append(tmp)
rusher_box = pd.concat(rusher_box_list)
rusher_box['Pass Rush_nflId'] = rusher_box['Pass Rush_nflId'].astype(int)

Pass Rush_#1
Pass Rush_#2
Pass Rush_#3
Pass Rush_#4
Pass Rush_#5
Pass Rush_#6
Pass Rush_#7


In [26]:
tmp = rusher_box.groupby(['gameId','playId'])[['frameId']].max().reset_index()
tmp['half_frameId'] = tmp['frameId'] // 2
rusher_box = pd.merge(
    rusher_box,
    tmp[['gameId','playId','half_frameId']],
    on=['gameId','playId'],
    how='left'
)
rusher_box_agg = rusher_box.groupby(['gameId','playId','Pass Rush_nflId'])[['blockers_ttl_impact_to_rusher']].mean().rename(columns={
    'blockers_ttl_impact_to_rusher':'blockers_ttl_impact_to_rusher_mean'})
rusher_box_agg['blockers_ttl_impact_to_rusher_sum'] = rusher_box.groupby(['gameId','playId','Pass Rush_nflId'])['blockers_ttl_impact_to_rusher'].sum()
rusher_box_agg = rusher_box_agg.reset_index()
rusher_box_agg.to_csv('../generated_data/blckr_impct_to_rshr{}_v6.csv'.format(_TGT_WEEK_START), index=False)

rusher_box = rusher_box[rusher_box['frameId'] <= rusher_box['half_frameId']]
rusher_box_agg = rusher_box.groupby(['gameId','playId','Pass Rush_nflId'])[['blockers_ttl_impact_to_rusher']].mean().rename(columns={
    'blockers_ttl_impact_to_rusher':'blockers_ttl_impact_to_rusher_mean'})
rusher_box_agg['blockers_ttl_impact_to_rusher_sum'] = rusher_box.groupby(['gameId','playId','Pass Rush_nflId'])['blockers_ttl_impact_to_rusher'].sum()
rusher_box_agg = rusher_box_agg.reset_index()
rusher_box_agg.to_csv('../generated_data/blckr_impct_to_rshr{}_firstHalf_v6.csv'.format(_TGT_WEEK_START), index=False)


## Blocker assignments アニメーション用データ出力 (week8のみ）

In [27]:
if _TGT_WEEK_START == 8:
    df[df['gameId_playId']=='2021102800_189'].to_csv('../to_Nao/blckr2rshr_impact_2021102800_189.csv', index=False)
    rusher_box[(rusher_box['gameId']==2021102800)&(rusher_box['playId']==189)].to_csv('../to_Nao/blckr_assignments_2021102800_189.csv', index=False)

## ブロッカー貢献度の計算

In [28]:
for j in range(1, 9+1):
    rusher_box['Pass Block_#{}_rshr_assgn_ratio'.format(j)] = rusher_box['Pass Block_#{}_impact_to_rshr'.format(j)] / rusher_box['blockers_ttl_impact_to_rusher']

cols = ['Pass Block_#{}_impact_to_rshr'.format(j) for j in range(1,9+1)] + ['Pass Block_#{}_nflId'.format(j) for j in range(1,9+1)]
blckr_assgn = rusher_box[rusher_box['Pass Rush_nflId'] > 0].groupby(['gameId','playId','Pass Rush_nflId'])[cols + ['blockers_ttl_impact_to_rusher']].mean().reset_index()

for j in range(1, 9+1):
    blckr_assgn['Pass Block_#{}_rshr_assgn_ratio'.format(j)] = blckr_assgn['Pass Block_#{}_impact_to_rshr'.format(j)] / blckr_assgn['blockers_ttl_impact_to_rusher']

blckr_assgn_concat = []
for j in range(1, 9+1):
    col = 'Pass Block_#{}'.format(j)
    blckr_assgn_concat.append(
        blckr_assgn[['gameId','playId','Pass Rush_nflId',col + '_rshr_assgn_ratio', col + '_nflId']].rename(columns={
            col + '_rshr_assgn_ratio': 'Blckr_assgn_ratio',
            col + '_nflId': 'Pass Block_nflId'
        })
    )
    
# Rusher目線からのassign ratioからBlocker目線からのassign ratioに更新
blckr_assgn2 = pd.concat(blckr_assgn_concat)
blckr_assgn2 = blckr_assgn2[blckr_assgn2['Pass Block_nflId'] > 0]
blckr_assgn2['Pass Block_nflId'] = blckr_assgn2['Pass Block_nflId'].astype(int)
blckr_assgn2 = pd.merge(
    blckr_assgn2,
    blckr_assgn2.groupby(['gameId','playId','Pass Block_nflId'])[['Blckr_assgn_ratio']].sum().reset_index().rename(columns={
    'Blckr_assgn_ratio': 'sum_Blckr_assgn_ratio'
    }),
    on=['gameId','playId','Pass Block_nflId'],
    how='left'
)
blckr_assgn2['Blckr_assgn_ratio2'] = blckr_assgn2['Blckr_assgn_ratio'] / blckr_assgn2['sum_Blckr_assgn_ratio']

rshr_impct_stats = []

for i in range(1, 7+1):
    col = 'Pass Rush_#{}'.format(i)
    rshr_impct_stats.append(
        play_result3[['gameId','playId',col + '_nflId', col + '_impact']].rename(columns={
            col + '_nflId': 'Pass Rush_nflId',
            col + '_impact': 'Pass Rush_impact'
        })
    )
rshr_impct_stats = pd.concat(rshr_impct_stats)
rshr_impct_stats = rshr_impct_stats[rshr_impct_stats['Pass Rush_nflId'] > 0]
blckr_assgn3 = pd.merge(
    blckr_assgn2,
    rshr_impct_stats,
    on=['gameId','playId','Pass Rush_nflId'],
    how='left'
)
blckr_assgn3 = blckr_assgn3[~blckr_assgn3['Pass Rush_impact'].isnull()]
blckr_assgn3['Weighted_PassRush_impact'] = blckr_assgn3['Pass Rush_impact'] * blckr_assgn3['Blckr_assgn_ratio2']
blckr_assgn4 = blckr_assgn3.groupby(['gameId','playId','Pass Block_nflId'])[['Weighted_PassRush_impact']].sum().reset_index()
blckr_assgn4['Blocker_contribution'] = 1 / (1 + blckr_assgn4['Weighted_PassRush_impact'])
blckr_assgn4.to_csv('../generated_data/blckr_contribution{}_v6.csv'.format(_TGT_WEEK_START), index=False)
